In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
import numpy as np

In [ ]:
# Ruta al archivo ZIP
zip_file_path = '/content/mascotas.zip'
# Carpeta donde quieres descomprimirlo
extract_path = '/content/mascotas'

# Comando para descomprimir
!unzip -q "{zip_file_path}" -d "{extract_path}"

print(f"Archivos descomprimidos en {extract_path}")


In [ ]:
# Configura el generador de imágenes
datagen = ImageDataGenerator(
    rotation_range=40,  # Permite rotar las imágenes hasta 40 grados
    width_shift_range=0.2,  # Desplaza horizontalmente las imágenes hasta un 20% del ancho
    height_shift_range=0.2,  # Desplaza verticalmente las imágenes hasta un 20% de la altura
    shear_range=0.2,  # Aplica una transformación de corte (shear) hasta un 20%
    zoom_range=0.2,  # Permite un zoom de hasta un 20% hacia adentro o hacia afuera
    horizontal_flip=True,  # Invierte horizontalmente las imágenes
    fill_mode='nearest'  # Rellena los nuevos píxeles utilizando el valor más cercano
)

In [ ]:
train_generator = datagen.flow_from_directory(
    '/content/mascotas',
    target_size=(300,300),  # Tamaño de destino de las imágenes
    class_mode='categorical',
    shuffle=False
)

In [ ]:
array_to_img(train_generator[0][0][0])

In [ ]:
array_to_img(train_generator[0][0][0])

In [ ]:
# Rutas de origen y destino
dataset_dir = '/content/mascotas'  # Ruta principal del dataset
augmented_dir = extract_path  # Ruta para las imágenes aumentadas

In [ ]:
def augment_images(src_dir, dest_dir):
    # Recorre las carpetas de conjuntos (test, validation, train)
    for subset in os.listdir(src_dir):
        subset_path = os.path.join(src_dir, subset)

        if os.path.isdir(subset_path):  # Verifica que sea un directorio
            # Crea el directorio correspondiente en el destino
            dest_subset_path = os.path.join(dest_dir, subset)
            os.makedirs(dest_subset_path, exist_ok=True)

            # Solo aplica aumento de datos si es la carpeta 'train'
            if subset == 'train':
                # Recorre las carpetas de clases dentro del conjunto
                for class_name in os.listdir(subset_path):
                    class_path = os.path.join(subset_path, class_name)

                    if os.path.isdir(class_path):  # Asegúrate de procesar solo carpetas de clases
                        os.makedirs(os.path.join(dest_subset_path, class_name), exist_ok=True)
                        for img_name in os.listdir(class_path):
                            img_path = os.path.join(class_path, img_name)
                            if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                                img = tf.keras.preprocessing.image.load_img(img_path)
                                img_array = tf.keras.preprocessing.image.img_to_array(img)
                                img_array = np.expand_dims(img_array, axis=0)

                                # Generar y guardar imágenes aumentadas
                                i = 0
                                # Cambia el número de imágenes a generar para la clase 'parrots'
                                num_images_to_generate = 14 if class_name.lower() == 'parrots' else 4

                                for batch in datagen.flow(img_array, batch_size=1,
                                                          save_to_dir=os.path.join(dest_subset_path, class_name),
                                                          save_prefix='aug', save_format='jpeg'):
                                    i += 1
                                    if i >= num_images_to_generate:  # Genera el número especificado de imágenes
                                        break
                                print(f"Generadas {i} imágenes aumentadas para {img_name} en {class_name}/{subset}")

            else:
                # Para las carpetas 'test' y 'validation', simplemente copia las imágenes
                for class_name in os.listdir(subset_path):
                    class_path = os.path.join(subset_path, class_name)
                    if os.path.isdir(class_path):
                        os.makedirs(os.path.join(dest_subset_path, class_name), exist_ok=True)
                        for img_name in os.listdir(class_path):
                            img_path = os.path.join(class_path, img_name)
                            if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                                # Copia las imágenes sin aumentarlas
                                tf.keras.preprocessing.image.save_img(
                                    os.path.join(dest_subset_path, class_name, img_name),
                                    tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(img_path))
                                )
                                print(f"Copiada imagen {img_name} en {class_name}/{subset}")

In [ ]:
# Ejecuta la función
augment_images(dataset_dir, augmented_dir)

In [ ]:
from google.colab import files
!zip -r mascotasDA.zip mascotasDA
files.download('mascotasDA.zip')